In [5]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [6]:
#  Your Spotify API credentials
SPOTIFY_CLIENT_ID = "b5a575454afa4804848c14862b2222ae"
SPOTIFY_CLIENT_SECRET = "c2ebbef0a6c6443ab4f5572b4d8e4e0e"
SPOTIFY_REDIRECT_URI = "http://127.0.0.1:8080/callback"

In [7]:
#  Authenticate
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri=SPOTIFY_REDIRECT_URI,
    scope="playlist-read-private"
))

In [8]:
#  Fetch all playlists of the user
playlists = sp.current_user_playlists()

In [9]:

#  Print Playlist Names & IDs
for playlist in playlists["items"]:
    print(f" {playlist['name']} → {playlist['id']}")


 Hymn🌥️🩶 → 0HzA2RquXyOHmqLIU75INx
 Feels 🤍 → 09EVxjpksf1uBjIJeeu8Qo
 Retro🌙☁️ → 6aljHh88xBsFDcd2G4fuWw


In [10]:
import os
import spotipy
import librosa
import numpy as np
import pandas as pd
import requests
import yt_dlp
from spotipy.oauth2 import SpotifyOAuth
from concurrent.futures import ThreadPoolExecutor

In [11]:
# Fetch Songs from Spotify Playlists 
playlist_tracks = []

playlists = sp.current_user_playlists()
for playlist in playlists['items']:
    playlist_name = playlist['name']
    playlist_id = playlist['id']
    results = sp.playlist_tracks(playlist_id)

    for item in results['items']:
        track = item['track']
        playlist_tracks.append({
            "playlist": playlist_name,
            "track_name": track['name'],
            "artist": track['artists'][0]['name'],
            "preview_url": track['preview_url']
        })

df_tracks = pd.DataFrame(playlist_tracks)
df_tracks.to_csv("spotify_tracks.csv", index=False)
print(f" Found {len(df_tracks)} tracks in your Spotify playlists.")

 Found 121 tracks in your Spotify playlists.


In [13]:
# STEP 3: Download Full Songs from YouTube 
download_dir = "full_songs"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

def sanitize_filename(name):
    """Convert track name to a safe filename."""
    return "".join(c if c.isalnum() else "_" for c in name)

def download_song(track_name, artist):
    search_query = f"{track_name} {artist} official audio"
    output_path = os.path.join(download_dir, f"{sanitize_filename(track_name)}.mp3")

    if os.path.exists(output_path):
        print(f" Already downloaded: {track_name}")
        return

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": os.path.join(download_dir, f"{sanitize_filename(track_name)}.webm"),
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }],
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([f"ytsearch1:{search_query}"])
            print(f" Downloaded: {track_name} (YouTube)")
        except Exception as e:
            print(f" Failed to download {track_name}: {e}")


# Download all songs
for _, row in df_tracks.iterrows():
    download_song(row["track_name"], row["artist"])

[youtube:search] Extracting URL: ytsearch1:Love Yourself Justin Bieber official audio
[download] Downloading playlist: Love Yourself Justin Bieber official audio
[youtube:search] query "Love Yourself Justin Bieber official audio": Downloading web client config
[youtube:search] query "Love Yourself Justin Bieber official audio" page 1: Downloading API JSON
[youtube:search] Playlist Love Yourself Justin Bieber official audio: Downloading 1 items of 1
[download] Downloading item 1 of 1
[youtube] Extracting URL: https://www.youtube.com/watch?v=WJUe3mr2gNM
[youtube] WJUe3mr2gNM: Downloading webpage
[youtube] WJUe3mr2gNM: Downloading tv client config
[youtube] WJUe3mr2gNM: Downloading player ab11fe69-main
[youtube] WJUe3mr2gNM: Downloading tv player API JSON
[youtube] WJUe3mr2gNM: Downloading ios player API JSON
[youtube] WJUe3mr2gNM: Downloading m3u8 information
[info] WJUe3mr2gNM: Downloading 1 format(s): 251
[download] Destination: full_songs/Love_Yourself.webm
[download] 100% of    3.75M

[info] 2pOVz45p8Ho: Downloading 1 format(s): 251
[download] Destination: full_songs/Adi_Penne__Duet____From_Naam_Series.webm
[download] 100% of    5.03MiB in 00:00:00 at 5.93MiB/s     
[ExtractAudio] Destination: full_songs/Adi_Penne__Duet____From_Naam_Series.mp3
Deleting original file full_songs/Adi_Penne__Duet____From_Naam_Series.webm (pass -k to keep)
[download] Finished downloading playlist: Adi Penne (Duet) - From Naam Series Stephen Zechariah official audio
 Downloaded: Adi Penne (Duet) - From Naam Series (YouTube)
[youtube:search] Extracting URL: ytsearch1:Idhazhin Oram - The Innocence of Love Anirudh Ravichander official audio
[download] Downloading playlist: Idhazhin Oram - The Innocence of Love Anirudh Ravichander official audio
[youtube:search] query "Idhazhin Oram - The Innocence of Love Anirudh Ravichander official audio": Downloading web client config
[youtube:search] query "Idhazhin Oram - The Innocence of Love Anirudh Ravichander official audio" page 1: Downloading API 

In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Path to your folder with MP3 songs
audio_dir = "full_songs"

# List all MP3 files
mp3_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith(".mp3")]

# Function to extract audio features
def extract_audio_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=22050, duration=30)

        # Beat-based Tempo (for reference)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr, onset_envelope=onset_env)

        # Danceability: Beat strength + tempo consistency
        danceability = np.mean(onset_env)

        # Energy: Use root-mean-square energy (volume)
        rms = librosa.feature.rms(y=y)
        energy = np.mean(rms)

        # Valence approximation: based on harmonic vs percussive content
        harmonic, percussive = librosa.effects.hpss(y)
        valence = np.mean(harmonic) / (np.mean(percussive) + 1e-6)

        return {
            "track_name": os.path.basename(file_path).replace(".mp3", ""),
            "danceability": danceability,
            "energy": energy,
            "valence": valence,
            "tempo": round(float(tempo.item()), 2)
        }

    except Exception as e:
        print(f" Error processing {file_path}: {e}")
        return None

# Extract features
audio_features = []
for file in mp3_files:
    print(f"Processing: {file}")
    result = extract_audio_features(file)
    if result:
        audio_features.append(result)

# Create DataFrame
df = pd.DataFrame(audio_features)

# Normalize danceability, energy, valence to 0–1
scaler = MinMaxScaler()
df[["danceability", "energy", "valence"]] = scaler.fit_transform(df[["danceability", "energy", "valence"]])

# Save to CSV
df.to_csv("audio_features_normalized.csv", index=False)

print("\n Feature extraction and normalization completed!")
print(df.head())


Processing: full_songs/Adiye___From__Bachelor_.mp3
Processing: full_songs/Mercy.mp3
Processing: full_songs/Innum_Konjam_Naeram.mp3
Processing: full_songs/Aaathi_Adi_Aaathi.mp3
Processing: full_songs/Mannipaaya.mp3
Processing: full_songs/Amma_Amma__From__Velai_Illa_Pattadhaari__.mp3
Processing: full_songs/Wildest_Dreams.mp3
Processing: full_songs/Vaa_Vaa_Anbe.mp3
Processing: full_songs/Ilaya_Nila.mp3
Processing: full_songs/One_Last_Time.mp3
Processing: full_songs/Roja_Kadale.mp3
Processing: full_songs/Naan_Sonnadhum_Mazhai.mp3
Processing: full_songs/Orey_Oru.mp3
Processing: full_songs/Blank_Space.mp3
Processing: full_songs/Thee_Illai.mp3
Processing: full_songs/Calm_Down__with_Selena_Gomez_.mp3
Processing: full_songs/Yennachu_Yedhachu.mp3
Processing: full_songs/Maari_s_Aanandhi.mp3
Processing: full_songs/The_Romance_Of_Power_Paandi___Venpani_Malare__Male_.mp3
Processing: full_songs/Thaniye__From__Vidaamuyarchi__.mp3
Processing: full_songs/Life_of_Pazham.mp3
Processing: full_songs/Yedho_O